# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime

In [3]:
def format_delta(delta: datetime.timedelta):
    hours, remainder = divmod(delta.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    h = f'{int(hours)}h ' if hours > 0 else ''
    m = f'{int(minutes)}m ' if minutes > 0 else ''
    s = f'{int(seconds)}s'
    
    return h+m+s

def estimate_time_to_finish(times: list, current_run_i: int, total_runs: int):
    deltas = []
    i = 1
    
    while i+1 <= len(times):
        deltas.append(times[i] - times[i-1])
        i += 1
        
    avg = sum(deltas, datetime.timedelta())/len(deltas)
    return f'Run {current_run_i}/{total_runs}. {format_delta(avg)} per run, estimate {format_delta(avg*(total_runs - current_run_i))} to finish'

In [4]:
adult = {
    "name": "adult",
    "path": "../datasets/adult/adult_dirty.csv",
    "clean_path": "../datasets/adult/adult.csv"
}
beers = {
    "name": "beers",
    "path": "../datasets/beers/dirty.csv",
    "clean_path": "../datasets/beers/clean.csv"
}
flights = {
    "name": "flights",
    "path": "../datasets/flights/dirty.csv",
    "clean_path": "../datasets/flights/clean.csv"
}
movies = {
    "name": "movies",
    "path": "../datasets/movies_1/dirty.csv",
    "clean_path": "../datasets/movies_1/clean.csv"
}
rayyan = {
    "name": "rayyan",
    "path": "../datasets/rayyan/dirty.csv",
    "clean_path": "../datasets/rayyan/clean.csv"
}
tax = {
    "name": "tax",
    "path": "../datasets/tax/dirty.csv",
    "clean_path": "../datasets/tax/clean.csv"
}
toy = {
    "name": "toy",
    "path": "../datasets/toy/dirty.csv",
    "clean_path": "../datasets/toy/clean.csv"
}
hospital = {
    "name": "hospital",
    "path": "../datasets/hospital/dirty.csv",
    "clean_path": "../datasets/hospital/clean.csv"
}

In [5]:
datasets = [adult]
results_no_update = []

data = raha.Dataset(beers)
n_pdep = 1

for dataset_dictionary in datasets:
    data = raha.Dataset(dataset_dictionary)
    data.detected_cells = dict(data.get_actual_errors_dictionary())
    n_pdep = 0
    while n_pdep < data.dataframe.shape[1]:
        app_2 = raha.Correction()
        app_2.LABELING_BUDGET = 20
        app_2.VERBOSE = False
        app_2.CLASSIFICATION_MODEL = "ABC"
        app_2.VICINITY_ONLY = True
        app_2.VICINITY_ORDER = 2
        app_2.HIGHER_ORDER_FEATURE_GENERATOR = 'pdep'
        app_2.N_BEST_PDEPS = n_pdep

        d = app_2.initialize_dataset(data)
        app_2.initialize_models(d)

        while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
            app_2.sample_tuple(d, random_seed=0)
            if d.has_ground_truth:
                app_2.label_with_ground_truth(d)
        app_2.update_models(d)
        app_2.generate_features(d, synchronous=False)
        app_2.predict_corrections(d, random_seed=0)

        p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
        results_no_update.append({'dataset': d.name, 'n_pdep': n_pdep, 'precision': p, 'recall': r, 'f1': f})
        print({'dataset': d.name, 'n_pdep': n_pdep, 'precision': p, 'recall': r, 'f1': f})
        
        n_pdep += 1

Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>
Failed to train an imputer model for column 1
<class 'datawig.autogluon_imputer.TargetColumnException'>
Failed to train an imputer model for column 2
<class 'datawig.autogluon_imputer.TargetColumnException'>
Failed to train an imputer model for column 4
<class 'datawig.autogluon_imputer.TargetColumnException'>
{'dataset': 'beers', 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>
Failed to train an imputer model for column 2
<class 'datawig.autogluon_imputer.TargetColumnException'>
{'dataset': 'flights', 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>
Failed to train an imputer model for column 1
<class 'datawig.autogluon_imputer.TargetColumnException'>
{'dataset': 'rayyan', 'precision': 0

		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"


{'dataset': 'hospital', 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}


In [ ]:
print(results_no_update)

[{'dataset': 'beers', 'n_pdep': 0, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, {'dataset': 'beers', 'n_pdep': 1, 'precision': 0.7, 'recall': 0.15887207702888584, 'f1': 0.2589686098654709}, {'dataset': 'beers', 'n_pdep': 2, 'precision': 0.5340030911901081, 'recall': 0.1584135717560752, 'f1': 0.24434229137199434}, {'dataset': 'beers', 'n_pdep': 3, 'precision': 0.4732770745428973, 'recall': 0.15428702430077945, 'f1': 0.23271092669432916}, {'dataset': 'beers', 'n_pdep': 4, 'precision': 0.4800824175824176, 'recall': 0.16024759284731774, 'f1': 0.2402887590237195}, {'dataset': 'beers', 'n_pdep': 5, 'precision': 0.4562334217506631, 'recall': 0.15772581384685924, 'f1': 0.23441226575809201}, {'dataset': 'beers', 'n_pdep': 6, 'precision': 0.44671052631578945, 'recall': 0.15566254011921138, 'f1': 0.23087385243114586}, {'dataset': 'beers', 'n_pdep': 7, 'precision': 0.4547231270358306, 'recall': 0.16001834021091244, 'f1': 0.23673054095302698}, {'dataset': 'beers', 'n_pdep': 8, 'precision': 0.445721